In [2]:
import sys
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from pymongo import MongoClient

In [3]:
sys.path.append('../src')

from scraping_support import *

In [4]:
# opciones del driver
opciones = Options()

# quita la bandera de ser robot
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

# guardar las cookies
opciones.add_argument('user-data-dir=cookies')    # mantiene las coockies

# Discogs Scraping

### Entrada a El Artico Discos y preparación de scrapeo 250/250

In [4]:
url = 'https://www.discogs.com/es/seller/elarticodiscos/profile'

driver = webdriver.Chrome(options = opciones)

driver.get(url)

In [5]:
try:
    driver.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]').click()
except:
    print('No necesario')

No necesario


In [6]:
# de 250 en 250
try:
    select = Select(driver.find_element(By.ID,'limit_top'))

    select.select_by_value('250')

except:
    print('Casi majo...')

### Creación de base de datos MongoDB

In [5]:
# Establecemos el cursor con la string de conexión por defecto

str_conn = 'mongodb://localhost:27017'   # str_con por defecto

cursor = MongoClient(str_conn)

cursor

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [6]:
# Creamos la base de datos

SonicHarbor = cursor.sonic_harbor

# Creamos la colección (similar a tabla SQL)

inventario = SonicHarbor.inventario

### Obtención de datos

In [9]:
for i in range(40):
    
    print(f'página {i}')
    
    time.sleep(3)
    
    # Limpia las listas en cada iteración
    item_description = []
    price_description = []        
        
# Obtención de datos
    try: 
        item_description = item(driver)
        price_description = price(driver)
    except:
        print(f'Fallo extracción {i}')
        
    
# Crear y preparar df
    try:
        df_item = pd.DataFrame(item_description)
        df_price = pd.DataFrame(price_description)

        #Concatenar los df, columnas e índice
        result = pd.concat([df_item, df_price], axis=1)
        result.columns = ['author', 'album', 'label', 'item_condition', 'sleeve_condition', 'price', 'ship_price', 'final_price']
    except:
        print(f'Fallo transformación {i}')

# Envió a mongo
    try:
        inventario.insert_many(result.to_dict(orient='records'))
    except:
        print(f'Fallo envió a Mongo {i}')
    
# Navegar a la siguiente página    
    try:
        next_page(driver)
    except:
        print(f'Fallo navegación a {i}')

página 0
página 1
página 2
página 3
página 4
página 5
página 6
página 7
página 8
página 9
página 10
página 11
página 12
página 13
página 14
página 15
página 16
página 17
página 18
página 19
página 20
página 21
página 22
página 23
página 24
página 25
página 26
página 27
página 28
página 29
página 30
página 31
página 32
página 33
página 34
página 35
página 36
página 37
página 38
página 39


Se realizan 40 iteraciones, lo que nos dará 10.000 items con información a lo largo de 8 columnas. Se decide no escrapear la totalidad de items debido a que no es necesario para la consecución de los objetivos de este proyecto.

### Cierre de Scrapeo

In [10]:
driver.quit()